In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn import linear_model

# data gathering & cleaning

In [2]:
nasdaq_100_tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST", "PDD",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "ZM", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "DOCU", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "ASAN", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "FOXA", "FOX", "ANSS",
    "SWKS", "OKTA", "TTD", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "PTON", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]
end_date = '2020-12-31'
start_date = '2016-3-28'
data = pd.DataFrame()
daily_log_ret = pd.DataFrame()
for ticker in nasdaq_100_tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data[ticker] = stock_data['Adj Close']
    data[ticker].index = data[ticker].index.strftime('%Y/%m/%d')
    daily_log_return = np.log(data[ticker] / data[ticker].shift(1))
    daily_log_ret[ticker] = daily_log_return


In [3]:
daily_log_ret=daily_log_ret.iloc[1: , :]

In [4]:
zeros=daily_log_ret.isnull().sum().to_frame()

In [5]:
zeros[zeros.iloc[:, 0]!=0]

,0
PDD,588
ZM,771
DOCU,526
ASAN,1137
FOXA,744
FOX,745
OKTA,261
TTD,124
PTON,882


In [6]:
drop_lst=['PDD', 'ZM', 'DOCU', 'ASAN', 'FOXA', 'FOX', 'OKTA', 'TTD', 'PTON']

In [7]:
daily_log_ret=daily_log_ret.drop(columns=drop_lst)

In [48]:
alpha=pd.read_csv('first round alpha.csv', header=0)
alpha=alpha.set_index(['ticker', 'Date'])

In [49]:
alpha

a49        a51        a12      a101       a10  \
ticker Date                                                              
AAPL   2016/03/29  -0.622499  -0.622499  -0.622499  0.957446  0.549451   
       2016/03/30  -0.469999  -0.469999  -0.469999  0.498901  0.241758   
       2016/03/31   0.142500   0.142500  -0.142500 -0.712891  0.593407   
       2016/04/01  -0.250000  -0.250000   0.250000  0.670730  0.472527   
       2016/04/04  -0.282501  -0.282501  -0.282501  0.363827  0.142857   
...                      ...        ...        ...       ...       ...   
ZM     2020/12/23  25.119995  25.119995  25.119995 -0.754339  0.990000   
       2020/12/24   8.709991   8.709991  -8.709991 -0.881488  0.990000   
       2020/12/28  23.779999  23.779999  23.779999 -0.783561  0.990000   
       2020/12/29  -2.359985  -2.359985   2.359985  0.316015  0.100000   
       2020/12/30   0.350006   0.350006  -0.350006 -0.377416  0.710000   

                        a18       a17       a46       a38       a34  ...  \
ticker Date                                                          ...   
AAPL   2016/03/29 -0.208791 -0.203893  0.622499 -0.165197  0.318681  ...   
       2016/03/30 -0.648352 -0.475400  1.000000 -0.580848  0.604396  ...   
       2016/03/31 -0.318681 -0.027071 -0.142500 -0.220807  0.445055  ...   
       2016/04/01 -0.340659 -0.163239  0.250000 -0.285956  0.615385  ...   
       2016/04/04 -0.802198 -0.631911  1.000000 -0.666526  0.510989  ...   
...                     ...       ...       ...       ...       ...  ...   
ZM     2020/12/23 -0.020000 -0.001287  1.000000 -0.015800  0.655000  ...   
       2020/12/24 -0.040000 -0.019206  1.000000 -0.000350  0.810000  ...   
       2020/12/28 -0.020000 -0.002489  1.000000 -0.001050  0.865000  ...   
       2020/12/29 -0.960000 -0.089743  1.000000 -0.046000  0.050000  ...   
       2020/12/30 -0.770000 -0.008649  3.914999 -0.009800  0.905000  ...   

                        a83       a47        a8       a16       a30       a45  \
ticker Date                                                                     
AAPL   2016/03/29  0.329670 -0.280757 -0.571429 -0.434066  0.137162 -0.162903   
       2016/03/30  0.131868 -0.444842 -0.604396 -0.445055  0.157253 -0.129915   
       2016/03/31  0.978022 -0.174318 -0.384615 -0.714286  0.196429 -0.096365   
       2016/04/01  0.120879 -0.269163 -0.461538 -0.637363  0.251621  0.172081   
       2016/04/04  0.087912 -0.299701 -0.549451 -0.197802  0.241054 -0.098660   
...                     ...       ...       ...       ...       ...       ...   
ZM     2020/12/23  0.510000  0.141713 -0.920000 -0.470000  0.071413  0.652500   
       2020/12/24  0.700000  0.068012 -0.060000 -0.610000  0.289387 -0.675000   
       2020/12/28  0.520000  0.208728 -0.020000 -0.190000  0.431617  0.666000   
       2020/12/29  0.060000  0.123667 -0.020000 -0.220000  0.312416  0.702000   
       2020/12/30  0.770000  0.129963 -0.020000 -0.950000  0.740843 -0.693000   

                     a7       a14       a55       a60  
ticker Date                                            
AAPL   2016/03/29   1.0  0.131729 -0.000000 -0.026278  
       2016/03/30   NaN -0.270974 -0.000000 -0.025084  
       2016/03/31   1.0 -0.059218 -0.627054  0.011467  
       2016/04/01   1.0 -0.025932 -0.604207 -0.027473  
       2016/04/04   NaN -0.177212 -0.557475  0.015886  
...                 ...       ...       ...       ...  
ZM     2020/12/23  16.0 -0.004774  0.280261  0.011485  
       2020/12/24   1.0 -0.002961  0.483085  0.012970  
       2020/12/28  46.0  0.005598  0.604366  0.009604  
       2020/12/29   1.0  0.107946  0.549857  0.003861  
       2020/12/30   1.0  0.036829  0.722640 -0.024752  

[120000 rows x 27 columns]

In [50]:
alpha.isnull().sum()

a49        0
a51        0
a12     5787
a101    5778
a10     5787
a18     5859
a17     5985
a46     5787
a38     5859
a34     5814
a5      5859
a28     5985
a43     6120
a42     5778
a11     5805
a13     5814
a20     5787
a83     5832
a47     5949
a8      5904
a16     5814
a30     5949
a45     6761
a7       436
a14     5859
a55     6227
a60     5859
dtype: int64

In [51]:
alpha=alpha.ffill()

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
alpha.isin([np.inf, -np.inf]).sum()

a49      0
a51      0
a12      0
a101     0
a10      0
a18      0
a17      0
a46      0
a38      0
a34      0
a5       0
a28      0
a43      0
a42      0
a11      0
a13      0
a20      0
a83      0
a47      0
a8       0
a16      0
a30     96
a45      0
a7       0
a14      0
a55      0
a60      0
dtype: int64

In [65]:
scaler = StandardScaler().fit(alpha) 
alpha_nor = scaler.transform(alpha)

# Lasso

In [66]:
daily_log_ret.columns

Index(['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'TSLA', 'META', 'NVDA', 'PYPL',
       'NFLX', 'ASML', 'ADBE', 'INTC', 'CMCSA', 'CSCO', 'PEP', 'AVGO', 'TMUS',
       'COST', 'TXN', 'QCOM', 'AMAT', 'MU', 'AMGN', 'INTU', 'ISRG', 'CSX',
       'VRTX', 'JD', 'GILD', 'BIDU', 'MRVL', 'REGN', 'MDLZ', 'ADSK', 'ATVI',
       'BIIB', 'ILMN', 'LRCX', 'ADP', 'BKNG', 'MELI', 'KLAC', 'NXPI', 'MNST',
       'WDAY', 'ROST', 'KDP', 'EA', 'ALGN', 'ADI', 'IDXX', 'DXCM', 'XEL',
       'CTAS', 'EXC', 'MAR', 'SNPS', 'CDNS', 'CPRT', 'SGEN', 'SPLK', 'ORLY',
       'DLTR', 'MTCH', 'MCHP', 'INCY', 'PCAR', 'CTSH', 'FAST', 'VRSK', 'CHKP',
       'ANSS', 'SWKS', 'CDW', 'TEAM', 'WBA', 'LULU', 'PAYX', 'VRSN', 'AEP',
       'ZBRA', 'TCOM', 'NTES', 'BMRN', 'ULTA', 'EXPE', 'CSGP', 'SIRI', 'EBAY',
       'WDC'],
      dtype='object')

In [67]:
x=alpha.loc[['AAPL']]
y=daily_log_ret['AAPL']

In [68]:
x

a49       a51       a12      a101       a10       a18  \
ticker Date                                                                     
AAPL   2016/03/29 -0.622499 -0.622499 -0.622499  0.957446  0.549451 -0.208791   
       2016/03/30 -0.469999 -0.469999 -0.469999  0.498901  0.241758 -0.648352   
       2016/03/31  0.142500  0.142500 -0.142500 -0.712891  0.593407 -0.318681   
       2016/04/01 -0.250000 -0.250000  0.250000  0.670730  0.472527 -0.340659   
       2016/04/04 -0.282501 -0.282501 -0.282501  0.363827  0.142857 -0.802198   
...                     ...       ...       ...       ...       ...       ...   
       2020/12/23  0.919998  0.919998 -0.919998 -0.726833  0.610000 -0.580000   
       2020/12/24 -1.009995 -1.009995  1.009995  0.275304  0.240000 -0.710000   
       2020/12/28 -4.720001 -4.720001 -4.720001  0.704776  0.100000 -0.880000   
       2020/12/29  1.820007  1.820007 -1.820007 -0.714448  0.810000 -0.180000   
       2020/12/30  1.149994  1.149994 -1.149994 -0.717867  0.850000 -0.170000   

                        a17       a46       a38       a34  ...       a83  \
ticker Date                                                ...             
AAPL   2016/03/29 -0.203893  0.622499 -0.165197  0.318681  ...  0.329670   
       2016/03/30 -0.475400  1.000000 -0.580848  0.604396  ...  0.131868   
       2016/03/31 -0.027071 -0.142500 -0.220807  0.445055  ...  0.978022   
       2016/04/01 -0.163239  0.250000 -0.285956  0.615385  ...  0.120879   
       2016/04/04 -0.631911  1.000000 -0.666526  0.510989  ...  0.087912   
...                     ...       ...       ...       ...  ...       ...   
       2020/12/23 -0.072729  1.000000 -0.342350  0.285000  ...  0.920000   
       2020/12/24 -0.288745  1.000000 -0.476000  0.400000  ...  0.010000   
       2020/12/28 -0.434676  1.000000 -0.768950  0.100000  ...  0.230000   
       2020/12/29 -0.023166  1.000000 -0.069600  0.520000  ...  0.740000   
       2020/12/30 -0.141825  1.000000 -0.073350  0.935000  ...  0.800000   

                        a47        a8       a16       a30       a45    a7  \
ticker Date                                                                 
AAPL   2016/03/29 -0.280757 -0.571429 -0.434066  0.137162 -0.162903   1.0   
       2016/03/30 -0.444842 -0.604396 -0.445055  0.157253 -0.129915   1.0   
       2016/03/31 -0.174318 -0.384615 -0.714286  0.196429 -0.096365   1.0   
       2016/04/01 -0.269163 -0.461538 -0.637363  0.251621  0.172081   1.0   
       2016/04/04 -0.299701 -0.549451 -0.197802  0.241054 -0.098660   1.0   
...                     ...       ...       ...       ...       ...   ...   
       2020/12/23 -0.128029 -0.730000 -0.680000  0.030922 -0.352000   1.0   
       2020/12/24 -0.026341 -0.770000 -0.720000  0.060518  0.294800   1.0   
       2020/12/28 -0.402967 -0.770000 -0.760000  0.172157 -0.365200 -56.0   
       2020/12/29 -0.111643 -0.810000 -0.720000  0.125917 -0.356400 -42.0   
       2020/12/30 -0.024303 -0.360000 -0.670000  0.117927 -0.391600   1.0   

                        a14       a55       a60  
ticker Date                                      
AAPL   2016/03/29  0.131729 -0.000000 -0.026278  
       2016/03/30 -0.270974 -0.000000 -0.025084  
       2016/03/31 -0.059218 -0.627054  0.011467  
       2016/04/01 -0.025932 -0.604207 -0.027473  
       2016/04/04 -0.177212 -0.557475  0.015886  
...                     ...       ...       ...  
       2020/12/23 -0.321358 -0.066987  0.015446  
       2020/12/24 -0.018063 -0.081369  0.017921  
       2020/12/28 -0.003653 -0.430635 -0.022178  
       2020/12/29  0.038833 -0.150861  0.014653  
       2020/12/30  0.012544 -0.426794  0.012772  

[1200 rows x 27 columns]

In [69]:
y

Date
2016/03/29    0.023395
2016/03/30    0.017308
2016/03/31   -0.005216
2016/04/01    0.009133
2016/04/04    0.010221
                ...   
2020/12/23   -0.007001
2020/12/24    0.007683
2020/12/28    0.035141
2020/12/29   -0.013405
2020/12/30   -0.008563
Name: AAPL, Length: 1200, dtype: float64

In [92]:
clf = linear_model.Lasso(alpha=0.001)

In [93]:
clf.fit(x, y)

Lasso(alpha=0.001)

In [94]:
clf.coef_

array([-1.14948834e-02, -0.00000000e+00,  0.00000000e+00,  1.48851638e-03,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -4.04773340e-05,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -4.62350205e-06, -0.00000000e+00, -5.34554771e-05,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00])

In [95]:
y_pred=clf.predict(x)

In [96]:
clf.score(x, y)

0.8586038695123221

In [98]:
from sklearn.linear_model import LassoCV

In [99]:
reg = LassoCV(cv=5, random_state=0).fit(x, y)

In [101]:
reg.score(x, y)

0.8276253996861875

In [102]:
reg.coef_

array([-9.99413144e-03, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -5.52938034e-05, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -6.93953981e-05,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00])

In [103]:
reg.alpha_

0.0037703110704284987

In [74]:
x2=alpha.loc[['MSFT']]
y2=daily_log_ret['MSFT']

In [77]:
clf2 = linear_model.Lasso(alpha=0.001)
clf2.fit(x2, y2)

Lasso(alpha=0.001)

In [78]:
clf2.coef_

array([-6.28918543e-03, -1.10147916e-17,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -2.23169060e-05, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -3.35304854e-05,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00])

In [104]:
reg2 = LassoCV(cv=5, random_state=0).fit(x2, y2)
reg.coef_

array([-9.99413144e-03, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -5.52938034e-05, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -6.93953981e-05,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00])

In [106]:
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "ANSS",
    "SWKS", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]

In [128]:
result=pd.DataFrame(columns=alpha.columns)
for ticker in tickers:
    x=alpha.loc[[ticker]]
    y=daily_log_ret[ticker]
    reg = LassoCV(cv=5, random_state=0).fit(x, y)
    result.loc[ticker] = reg.coef_ 

In [129]:
result

,a49,a51,a12,a101,a10,a18,a17,a46,a38,a34,...,a83,a47,a8,a16,a30,a45,a7,a14,a55,a60
AAPL,-0.009994,-0.000000e+00,-0.000000,0.000000,-0.000000,0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.000000e+00,-0.0,-0.000069,-0.0,0.0,-0.0
MSFT,-0.005503,-1.080470e-18,-0.000000,0.000000,-0.000000,-0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,-0.000000e+00,-0.0,-0.000038,0.0,0.0,-0.0
AMZN,-0.007093,-1.807455e-18,-0.000000,0.000000,-0.000000,-0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,0.000000e+00,-0.0,-0.000057,-0.0,-0.0,-0.0
GOOGL,-0.014787,-4.489531e-18,-0.000000,0.000000,-0.000000,-0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,-8.285886e-07,-0.0,-0.000021,-0.0,-0.0,-0.0
GOOG,-0.014859,-1.798290e-17,-0.000000,0.000000,-0.000000,-0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,-0.000000e+00,-0.0,-0.000019,0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXPE,-0.010241,0.000000e+00,0.000976,-0.000919,0.000248,0.0,0.0,-0.000115,0.0,0.0,...,-0.0,0.0,0.0,-0.0,0.000000e+00,-0.0,-0.000006,-0.0,-0.0,0.0
CSGP,-0.013517,-1.035537e-18,-0.000000,0.000000,-0.000000,-0.0,-0.0,0.000000,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,-3.423618e-06,-0.0,-0.000065,-0.0,0.0,-0.0
SIRI,-0.124511,-1.355322e-16,-0.000000,0.005130,0.000000,0.0,-0.0,0.000000,-0.0,0.0,...,-0.0,-0.0,0.0,-0.0,0.000000e+00,-0.0,-0.000034,-0.0,-0.0,-0.0
EBAY,-0.023169,-2.138294e-18,-0.000000,0.000000,0.000000,0.0,-0.0,0.000000,-0.0,0.0,...,-0.0,-0.0,-0.0,-0.0,-0.000000e+00,-0.0,-0.000035,-0.0,0.0,-0.0


In [132]:
result.to_csv('lasso_alpha.csv')